# ASG Quant Fund Project

#### Armaan Gandhara

## Week 1

### Data Loader Development

In [103]:
import yfinance as yf
import pandas as pd
from typing import List, Union

In [104]:
class data_loader:

    def __init__(self):
        pass

    def get_data(self, ticker: str, start: str, end: str) -> pd.DataFrame:
        ticker = ticker.replace('.', '-')

        data = yf.download(ticker, start=start, end=end, progress=False)
        if data.empty:
            print(f"[!] Failed to download {ticker}. Skipping.")
            return None
        data.dropna(inplace=True)
        if 'Adj Close' in data.columns:
            data.drop(columns=['Adj Close'], inplace=True)
        #data.rename(columns={'Open': 'open','High': 'high','Low': 'low','Close': 'close','Volume': 'volume'}, inplace=True)

        #required_cols = ['open', 'high', 'low', 'close', 'volume']
        #data = data[required_cols]
        data = data.droplevel('Ticker', axis=1)
        data.reset_index(inplace=True)
        data.index = data['Date']
        del data['Date']
        data.index.name = 'Date'

        return data


    def get_multiple_data(self, tickers: List[str], start: str, end: str) -> dict:
        data_dict = {}
        for ticker in tickers:
            df = self.get_data(ticker, start, end)
            if df is not None:
                data_dict[ticker] = df

        return data_dict

### Strategy 1 Mean Reversion

#### Not Used

In [9]:
import pandas as pd
import pandas_ta as ta

c:\Users\agand\OneDrive\Desktop\ASG MICROFUND\venv\Lib\site-packages\pandas_ta\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [ ]:

class mean_reversion_strategy:
    def __init__(self, lookback: int = 20, std_dev: float = 2.0, threshold: float = 0.0):
        self.lookback = lookback
        self.std_dev = std_dev
        self.threshold = threshold

    def generate_signals(self, data: pd.DataFrame) -> pd.DataFrame:
        df = data.copy()

        bb = ta.bbands(close=df['Close'], length=self.lookback, std=self.std_dev)

        if bb is None or bb.empty:
            raise ValueError("Bollinger Bands calculation failed. Check input data.")

        df = df.join(bb)

        df['signal'] = 0  # Default to hold (0)
        
        df.loc[df['Close'] < df[f'BBL_{self.lookback}_{self.std_dev}'] * (1 - self.threshold), 'signal'] = 1
        
        df.loc[df['Close'] > df[f'BBU_{self.lookback}_{self.std_dev}'] * (1 + self.threshold), 'signal'] = -1

        return df[['signal']]


#### Used

In [ ]:
from backtesting import Strategy, Backtest
import pandas as pd
import pandas_ta as pdt

class mean_reversion_strategy(Strategy):
    length = 20
    std = 2.0
    def init(self):
        price = pd.Series(self.data.Close)
        bb = ta.bbands(close=price, length=self.length, std=self.std)

        self.lower = self.I(lambda: bb[f'BBL_{self.length}_{self.std}'])
        self.upper = self.I(lambda: bb[f'BBU_{self.length}_{self.std}'])

    def next(self):
        price = self.data.Close[-1]
        if price < self.lower[-1] and not self.position:
            self.buy(size=int(self.equity / price))
        elif price > self.upper[-1] and not self.position:
            self.sell(size=int(self.equity / price))

        if self.position.is_long and price > self.data.Close[-2]:
            self.position.close()
        elif self.position.is_short and price < self.data.Close[-2]:
            self.position.close()



### Backtesting Engine

In [57]:
class GenericBacktestEngine:
    def __init__(self, strategy_cls, strategy_kwargs: dict = None, cash: float = 10000, commission: float = 0.002):
        self.strategy_cls = strategy_cls
        self.strategy_kwargs = strategy_kwargs or {}
        self.cash = cash
        self.commission = commission

    def run(self, data: pd.DataFrame):
        bt = Backtest(
            data,
            self.strategy_cls,
            cash=self.cash,
            commission=self.commission
        )
        stats = bt.run(**self.strategy_kwargs)
        return stats

    def plot(self, data: pd.DataFrame):
        bt = Backtest(
            data,
            self.strategy_cls,
            cash=self.cash,
            commission=self.commission
        )
        bt.run(**self.strategy_kwargs)
        bt.plot()

    def batch_backtest(self, data_dict: dict):
        """
        Run backtests on a dict of ticker: DataFrame pairs.
        Returns a dict of ticker: stats
        """
        results = {}
        for ticker, data in data_dict.items():
            try:
                stats = self.run(data)
                results[ticker] = stats
            except Exception as e:
                print(f"Failed on {ticker}: {e}")
        return results


### Improving the Mean Reversion Strategy

In [ ]:
from backtesting import Strategy, Backtest
import pandas_ta as ta
class mean_reversion_strategy(Strategy):
    length = 20
    std = 2.0
    
    def init(self):
        price = pd.Series(self.data.Close)
        bb = ta.bbands(close=price, length=self.length, std=self.std)
        self.lower = self.I(lambda: bb[f'BBL_{self.length}_{self.std}'])
        self.upper = self.I(lambda: bb[f'BBU_{self.length}_{self.std}'])

    def next(self):
        price = self.data.Open[-1]
        # Entry
        if price < self.lower[-1]*1.2 and not self.position:
            self.buy(size=int(self.equity / price), sl=(price*0.90), limit=price*0.95)
        elif price > self.upper[-1] and not self.position:
            self.sell(size=int(self.equity / price))

        # Exit
        if self.position.is_long and price >= self.upper[-1]:
            self.position.close()
        elif self.position.is_short and price <= self.lower[-1]:
            self.position.close()

In [58]:
dt = data_loader()
data = dt.get_data("TSLA", '2023-01-01', '2025-06-01')

#Fix index issues
#data.index = data['Date']
#del data['Date']
#data.index.name = 'Date'

engine = GenericBacktestEngine(
    strategy_cls=mean_reversion_strategy,
    strategy_kwargs={'length': 20, 'std': 2.0},
    cash=10000,
    commission=0.000
)

results = engine.run(data)
engine.plot(data)
print(results)


c:\Users\agand\OneDrive\Desktop\ASG MICROFUND\venv\Lib\site-packages\backtesting\backtesting.py:1015: UserWarning: (2025-03-11 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
c:\Users\agand\OneDrive\Desktop\ASG MICROFUND\venv\Lib\site-packages\backtesting\backtesting.py:1015: UserWarning: (2025-03-11 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting

Start                     2023-01-03 00:00:00
End                       2025-05-30 00:00:00
Duration                    878 days 00:00:00
Exposure Time [%]                     76.3245
Equity Final [$]                  24054.21955
Equity Peak [$]                   24894.85929
Return [%]                           140.5422
Buy & Hold Return [%]               100.01154
Return (Ann.) [%]                    44.22484
Volatility (Ann.) [%]                87.73044
CAGR [%]                             28.64946
Sharpe Ratio                           0.5041
Sortino Ratio                         1.30548
Calmar Ratio                          1.00383
Alpha [%]                           130.00837
Beta                                  0.10533
Max. Drawdown [%]                    -44.0561
Avg. Drawdown [%]                    -9.63726
Max. Drawdown Duration      326 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                   32
Win Rate [%]                      

In [101]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500 = sp500['Symbol'].to_list()
tickers = []
sorter = True
for ticker in sp500:
    if sorter:
        tickers.append(ticker)
        
    #sorter = not sorter


In [105]:
data_loader = data_loader()
data = data_loader.get_multiple_data(tickers, '2023-01-01', '2025-06-01')

# Run batch test
engine = GenericBacktestEngine(mean_reversion_strategy)
results = engine.batch_backtest(data)

# Example: Print best performing stocks
sorted_results = sorted(results.items(), key=lambda x: x[1]['Return [%]'], reverse=True)
for ticker, stat in sorted_results[:10]:
    print(f"{ticker}: {stat['Return [%]']:.2f}%")


c:\Users\agand\OneDrive\Desktop\ASG MICROFUND\venv\Lib\site-packages\backtesting\backtesting.py:1015: UserWarning: (2025-04-07 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
c:\Users\agand\OneDrive\Desktop\ASG MICROFUND\venv\Lib\site-packages\backtesting\backtesting.py:1015: UserWarning: (2025-04-09 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting

MPWR: 132.47%
LII: 110.41%
TPL: 95.95%
CRWD: 94.00%
INTU: 92.07%
NOW: 83.37%
MCK: 77.49%
AZO: 72.43%
ISRG: 68.93%
NFLX: 67.76%


In [106]:
num = 0
total = 0
for ticker, stat in sorted_results:
    if stat['Return [%]']!=0:
        total += stat['Return [%]']
        num += 1
        print(f"{ticker}: {stat['Return [%]']:.2f}%")

MPWR: 132.47%
LII: 110.41%
TPL: 95.95%
CRWD: 94.00%
INTU: 92.07%
NOW: 83.37%
MCK: 77.49%
AZO: 72.43%
ISRG: 68.93%
NFLX: 67.76%
MCO: 66.08%
SNPS: 61.07%
META: 58.01%
CEG: 57.25%
RCL: 55.87%
MSCI: 54.77%
KLAC: 52.94%
CPAY: 52.43%
BLK: 52.31%
GS: 51.76%
ORLY: 48.30%
MA: 45.82%
MSFT: 45.79%
HD: 45.58%
GE: 45.31%
AXP: 43.98%
CDNS: 43.90%
TT: 43.87%
MSI: 41.83%
ADSK: 40.44%
RL: 39.92%
ANSS: 38.81%
BKNG: 36.41%
ZBRA: 35.77%
BRK.B: 35.24%
ROP: 34.13%
NOC: 32.87%
VMC: 32.50%
ACN: 31.27%
VRTX: 28.54%
CB: 28.36%
ESS: 28.05%
AXON: 26.73%
GWW: 25.75%
PH: 25.18%
NVR: 25.15%
VRSK: 25.09%
TDY: 24.29%
EQIX: 23.40%
DPZ: 23.05%
PODD: 22.57%
IDXX: 22.54%
AAPL: 22.43%
ETN: 22.18%
CMI: 22.04%
WDAY: 20.84%
GEV: 18.19%
DE: 18.16%
PAYC: 17.28%
WAT: 16.07%
FFIV: 16.06%
ECL: 15.88%
SYK: 15.45%
MOH: 15.30%
RMD: 15.09%
LH: 15.08%
PSA: 14.79%
TDG: 14.40%
CME: 14.05%
MTD: 13.80%
CHTR: 13.75%
CAT: 13.71%
POOL: 13.60%
PWR: 13.48%
UHS: 12.10%
TRV: 11.74%
ANET: 11.46%
GRMN: 11.29%
GD: 10.94%
COIN: 10.80%
RSG: 10.53%
CRM

In [ ]:
ave = total/num
ave

9.671420751152958

## Week 2